In [1]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

import numpy as np
import pandas as pd
import cv2, glob, os, torch
import torch.nn as nn
import torch.optim as optim
import torchvision

from torch.utils.data import Dataset, Subset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from albumentations.augmentations.geometric.transforms import PadIfNeeded
from torchvision.models import efficientnet_b0

In [2]:
# global variables
root_dir = './datasets/'
labelencoder = LabelEncoder()

In [3]:
# pandas 정리
train_label_frame = pd.read_csv(root_dir + 'train.csv', index_col=1)
test_index_file_frame = pd.read_csv(root_dir + 'test.csv', index_col=1)
artist_info_frame = pd.read_csv(root_dir + 'artists_info.csv')

# 데이터 전처리

In [4]:
class ArtworkTrainDataset(Dataset):

    def __init__(self, root_dir, file_label_frame):
        self.root_dir = root_dir
        self.file_label_frame = file_label_frame
        if self.file_label_frame['artist'].dtype == np.object0:
            raise TypeError('라벨값을 정수로 인코딩한 후에 적제해주세요.')
        self.padder = PadIfNeeded(min_height=None,
                                  min_width=None,
                                  pad_height_divisor=10,
                                  pad_width_divisor=10,
                                  border_mode=cv2.BORDER_CONSTANT
                                 )

    def __len__(self):
        return len(self.file_label_frame.index)

    def __getitem__(self, idx):
        image_path = self.file_label_frame.index[idx]
        image_label = self.file_label_frame.loc[image_path, 'artist']
        image = cv2.imread(self.root_dir + image_path)
        padded = self.padder(image=image)['image']
        transform = A.Compose([A.Resize(224,224), A.Normalize(), ToTensorV2()])
        
        w = padded.shape[0]
        h = padded.shape[1]
        # layer2
        crops_layer2 =torch.cat([
                               transform(image=padded[:w//2, :h//2, :])['image'],
                               transform(image=padded[:w//2, h//2:, :])['image'],
                               transform(image=padded[w//2:, :h//2, :])['image'],
                               transform(image=padded[w//2:, h//2:, :])['image']
        ], dim=0)
        crops_layer3 = []
        for i in range(5):
            for j in range(5):
                start_w, start_h = i * (w // 5), j * (h // 5)
                end_w, end_h = (i + 1) * (w // 5), (j + 1) * (h // 5)
                crops_layer3.append(transform(image=padded[start_w:end_w, start_h:end_h, :])['image'])
        crops_layer3 = torch.cat(crops_layer3, dim=0)
        
        image = torch.Tensor(transform(image=padded)['image'])
        return torch.cat([image, crops_layer2, crops_layer3], dim=0).reshape(30, 3, 224, 224), image_label

In [5]:
train_label_frame['artist'] = labelencoder.fit_transform(train_label_frame['artist'])
train = ArtworkTrainDataset(root_dir, train_label_frame)

# 모델

In [6]:
class PyramidNet(nn.Module):
    def __init__(self, baseline, baseline_kwargs, num_baseline_output, num_classes):
        super().__init__()
        self.network = nn.Sequential(
            baseline(**baseline_kwargs),
            nn.ReLU(),
            nn.Linear(in_features=num_baseline_output, out_features=num_classes)
        )

    def __call__(self, x):
        return self.forward(x)

    def forward(self, x):
        batch_size = x[0].size()[0]
        rest = x[0].size()[1:3]
        softmax = nn.Softmax(dim=0)
        x = self.network(x)
        layers = torch.split(x, split_size_or_sections=[1, 4, 25])
        
        return softmax(25 * layers[0].flatten()) + 4 * softmax(layers[1]).mean(dim=0) + softmax(layers[2]).mean(dim=0)

In [7]:
model = PyramidNet(efficientnet_b0, {'weights':torchvision.models.EfficientNet_B0_Weights.DEFAULT}, 1000, 50)

# 훈련

In [8]:
learning_rate = 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
epochs = 30

In [9]:
train_idx, valid_idx = train_test_split(range(len(train)), test_size=0.2, random_state=12345, stratify=train_label_frame['artist'])

In [10]:
trainset = Subset(train, train_idx)
validset = Subset(train, valid_idx)

In [ ]:
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for data, label in trainset:
        pred = model(data)
        label_t = torch.zeros(50)
        label_t[label] = 1
        loss = loss_fn(pred, label_t)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    model.eval()
    valid_loss = 0
    for data, label in validset:
        pred = model(data)
        label_t = torch.zeros(50)
        label_t[label] = 1
        loss = loss_fn(pred, label_t)
        valid_loss += loss.item()
    train_loss /= len(trainset)
    valid_loss /= len(validset)
    scheduler.step(valid_loss)
    print(f'Epoch {epoch + 1}, Train Loss: {train_loss}, Valid Loss: {valid_loss}')